In [ ]:
import pandas as pd

# --- Load data ---
df_1m = pd.read_csv("./data/agg_data/fx/C:EURUSD_1m_last1y.csv", parse_dates=["timestamp"])


In [1]:
import pandas as pd
from pathlib import Path
from collections import deque
from tqdm import tqdm

from pipeline.generate_image import ImageGenerator
from pipeline.zone_engine import ZoneEngine
from pipeline.aggregator import TimeframeAggregator

# --- Config ---
csv_path = "./data/agg_data/fx/C:EURUSD_1m_last1y.csv"
save_dir = Path("./dataset")
base_tf = "1m"
timeframes = ["1m", "3m", "5m", "15m", "1h", "4h", "1d"]
candle_limits = {tf: 60 for tf in timeframes}
image_size = (640, 640)
preload_days = 60

# --- Setup ---
df = pd.read_csv(csv_path, parse_dates=["timestamp"])
buffer = deque(maxlen=1000)
aggregator = TimeframeAggregator(buffer)
zone_engine = ZoneEngine(timeframes)
image_gen = ImageGenerator(candle_limits, image_size=image_size)

# --- Meta output ---
rows = []

# --- Main loop ---
for i in tqdm(range(len(df))):
    bar = df.iloc[i].to_dict()
    buffer.append(bar)

    if len(buffer) < 100:
        continue  # not enough data

    resampled = aggregator.resample_all(timeframes)

    for tf in timeframes:
        if len(resampled[tf]) < candle_limits[tf]:
            continue

        last_bar = resampled[tf].iloc[-1].to_dict()
        zone_engine.update(tf, last_bar)

    # Skip preload period
    if df.iloc[i]["timestamp"] < df.iloc[0]["timestamp"] + pd.Timedelta(days=preload_days):
        continue

    timestamp = df.iloc[i]["timestamp"]
    close_price = df.iloc[i]["close"]

    tf_image_paths = {}

    for tf in timeframes:
        df_tf = resampled[tf]
        if len(df_tf) < candle_limits[tf]:
            break

        zones = zone_engine.get_visible_zones(tf)
        img_path = save_dir / "images" / tf / f"{timestamp}.png"
        image_gen.generate_image(tf, df_tf, zones, str(img_path))

        tf_image_paths[f"{tf}_path"] = str(img_path)

    # Save metadata row
    row = {"timestamp": timestamp, "close": close_price}
    row.update(tf_image_paths)
    rows.append(row)

# --- Save metadata ---
meta_df = pd.DataFrame(rows)
meta_df.to_csv(save_dir / "meta.csv", index=False)
print(f"[DONE] Saved {len(meta_df)} samples with metadata.")


  2%|▏         | 6041/368307 [01:23<1:23:52, 71.99it/s]


KeyboardInterrupt: 

In [ ]:
# from env.trading_env import TradingEnv
# import pandas as pd

# # --- Load data ---
# df_1m = pd.read_csv("./data/agg_data/fx/C:EURUSD_1m_last1y.csv", parse_dates=["timestamp"])

# # Instantiate environment with your meta data
# env = TradingEnv(df_1m)

# # --- Step 1: Preload phase ---
# print("[PHASE 1] Preloading...")

# while not env.pipeline.preload_done and env.current_index < len(env.meta_df):
#     bar = env.meta_df.iloc[env.current_index].to_dict()
#     env.pipeline.process_new_candle(bar)
#     env.current_index += 1

# print(f"[DONE] Preload complete at index {env.current_index}")

# # --- Step 2: Real interaction starts here ---
# print("\n[PHASE 2] Stepping through environment...")

# done = False
# step = 0
# while not done and step < 100:
#     action = env.action_space.sample()
#     obs, reward, terminated, truncated, info = env.step(action)
#     done = terminated or truncated

#     realized_pnl = obs["features"][0]
#     unrealized_pnl = obs["features"][1]
#     position = obs["features"][2]
#     last_action = obs["features"][3]
#     steps_in_trade = obs["features"][4]
#     last_reward = obs["features"][5]

#     print(f"\nStep {step}")
#     print(f"  Action taken: {action}")
#     print(f"  Reward: {reward:.2f}")
#     print(f"  Realized PnL: {realized_pnl:.2f}")
#     print(f"  Unrealized PnL: {unrealized_pnl:.2f}")
#     print(f"  Position: {position}")
#     print(f"  Steps in trade: {int(steps_in_trade)}")
#     print(f"  Last Action: {int(last_action)}")
#     print(f"  Obs[images] shape: {obs['images'].shape}")
#     print(f"  Obs[features]: {obs['features']}")

#     step += 1


In [ ]:

print("Step obs shape:", obs["images"].shape)
print("Features:", obs["features"])


In [ ]:
# from pathlib import Path
# import pandas as pd
# from stable_baselines3 import PPO
# from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
# from stable_baselines3.common.monitor import Monitor

# from env.trading_env import TradingEnv
# from models.yolo_extractor import YOLOCNNExtractor

# # --- Load FX Data ---
# df_1m = pd.read_csv("./data/agg_data/fx/C:EURUSD_1m_last1y.csv", parse_dates=["timestamp"])


# def make_train_env():
#     env = TradingEnv(df_1m.copy())

#     # Preload pipeline
#     while not env.pipeline.preload_done and env.current_index < len(env.meta_df):
#         bar = env.meta_df.iloc[env.current_index].to_dict()
#         env.pipeline.process_new_candle(bar)
#         env.current_index += 1

#     return env


# # --- Build Vectorized Training Environment ---
# train_env = DummyVecEnv([make_train_env])
# train_env = VecNormalize(train_env, norm_obs=True, norm_reward=True, clip_obs=10.0)


# # --- PPO Model Configuration ---
# policy_kwargs = dict(
#     features_extractor_class=YOLOCNNExtractor,
#     features_extractor_kwargs=dict(
#         model_path="./models/best.pt",
#         features_dim=512,
#     ),
# )

# model = PPO(
#     policy="MultiInputPolicy",
#     env=train_env,
#     n_steps=128,
#     batch_size=32,
#     n_epochs=4,
#     learning_rate=3e-4,
#     policy_kwargs=policy_kwargs,
#     verbose=1,
#     device="cuda",
#     tensorboard_log="./tensorboard_logs/",
# )


# # --- Train the PPO Agent ---
# model.learn(total_timesteps=200_000)

# # --- Save Model + VecNormalize Stats ---
# model.save("./models/rl/final_model")
# train_env.save("./models/rl/vecnormalize.pkl")


Preloading 60 days: 100%|██████████| 86400/86400 [00:02<00:00, 35524.26min/s]


[INFO] Preload complete after 86400 minutes. Starting image generation.


INFO:env.trading_env:[ENV INIT] Preload complete. Start index set to 86400


Using cpu device
Logging to ./tensorboard_logs/PPO_9
----------------------------
| time/              |     |
|    fps             | 0   |
|    iterations      | 1   |
|    time_elapsed    | 149 |
|    total_timesteps | 128 |
----------------------------


KeyboardInterrupt: 